In [ ]:
!pip install llama-cpp-python

In [ ]:
!pip install llama-cpp-python[server]

In [ ]:
import requests
import os

url = 'https://huggingface.co/vinai/PhoGPT-4B-Chat-gguf/resolve/main/PhoGPT-4B-Chat-Q4_K_M.gguf?download=true'

save_directory = 'models' 
filename = 'PhoGPT-4B-Chat-Q4_K_M.gguf'

file_path = os.path.join(save_directory, filename)

if os.path.exists(file_path):
    print(f'Tệp {filename} đã tồn tại trong thư mục {save_directory}.')
else:
    response = requests.get(url)

    if response.status_code == 200:
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f'Tệp đã được tải xuống và lưu tại {file_path}')
    else:
        print('Có lỗi xảy ra trong quá trình tải xuống.')

### Run Llama.cpp server
**python -m llama_cpp.server --model models/PhoGPT-4B-Chat-Q4_K_M.gguf --n_ctx 8192**

### Call model via API (llama_cpp server) with Langchain

In [1]:
from langchain.llms import OpenAI
llm = OpenAI(base_url="http://localhost:8000/v1", api_key="llama.cpp")

c:\Users\Admin\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


### Define the Offline_RAG Class

In [8]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

class Offline_RAG:
    def __init__(self, llm) -> None:
        self.llm = llm
        self.messages = [
            (
                "system","Bạn là trợ lý AI hỗ trợ về sức khoẻ tâm lý sau sinh. Dựa vào ngữ cảnh bên dưới hãy đưa ra câu trả lời và lời khuyên phù hợp cho câu hỏi của họ. Nếu bạn không biết câu trả lời, hãy nói không biết, đừng cố tạo ra câu trả lời.\n# Ngữ cảnh: \n{context}\n"
            ),
            (
                "user","## Câu hỏi: {question}\n### Trả lời:"
            )
        ]

        self.prompt = ChatPromptTemplate.from_messages(self.messages)

    def get_chain(self, retriever):
        input_data = {
            "context": retriever | self.format_docs,
            "question": RunnablePassthrough()
        }
        rag_chain = (
            input_data
            | self.prompt
            | self.llm
        )
        return rag_chain
    
    def format_docs(self, docs):
        formatted_docs = "\n\n".join(doc.page_content for doc in docs)
        self.list_doc ={"context" :formatted_docs}
        print(self.list_doc)
        return formatted_docs

### Define the VectorDB class using Pinecone

In [3]:
import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore

class VectorDB:
    def __init__(self,
                 embedding = HuggingFaceEmbeddings()) -> None:
        os.environ['PINECONE_API_KEY'] = "73bdec39-1f93-47fc-bd2f-f02883d7be83"
        self.pinecone_api_key = os.getenv('PINECONE_API_KEY')
        self.embedding = embedding
        self.db = self._build_db()
    
    def _build_db(self):
        db = PineconeVectorStore.from_existing_index(
            index_name="docs-rag-chatbot",
            namespace="docs-search",
            embedding=self.embedding
        )
        return db
    
    # tính similarity
    def get_retriever(self,
                      search_type: str = "similarity",
                      search_kwargs: dict = {"k": 5}
                      ):
        retriever =self.db.as_retriever(search_type=search_type,
                                        search_kwargs=search_kwargs)
        return retriever

c:\Users\Admin\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


### Function build chain

In [4]:
def build_rag_chain(llm):
    retriever = VectorDB().get_retriever()
    off_rag = Offline_RAG(llm)
    rag_chain = off_rag.get_chain(retriever)
    return rag_chain, off_rag

def build_rag_chain_from_off_rag(off_rag):
    retriever = VectorDB().get_retriever()
    rag_chain = off_rag.get_chain(retriever)
    return rag_chain

### Run example

In [16]:
chain, off_rag = build_rag_chain(llm)
while True:
    question = input("You: ")
    if question == "exit":
        break
    answer = chain.invoke(question)
    for i, (role, text) in enumerate(off_rag.messages):
        if role == "user" and "{question}" in text:
            off_rag.messages[i] = (role, text.replace("{question}", question))
    off_rag.messages.append(("assistant", answer))
    off_rag.messages.append(("user","## Câu hỏi: {question}\n### Trả lời:"))
    chain = build_rag_chain_from_off_rag(off_rag)
    print(off_rag.messages)

{'context': 'khoẻ sinh sản cho phụ nữ bao gồm cả sàng lọc bạo lực và trầm cảm trong \nmang thai và sau sinh.\n\n- Tiền sử sảy thai \n- Có thai ngoài ý muốn \n- Sinh nhiều con \n \nYẾU TỐ SAU SINH \n- Chế độ nghỉ ngơi, kiêng khem sau \nsinh \n- Giới tính của trẻ \n- Thiếu hỗ trợ gia đình, xã hội sau \nsinh \n- Sinh non \n- Hình thức sinh: sinh mổ/sinh \nthường\n\ncác nhân viên y tế trong thời gian mang thai và sau sinh nhưng họ không tiết\n\nsinh con bác sĩ có thể kê đơn thuốc phòng ngừa trầm cảm sau sinh ngay sau khi em bé chào đời.\n\nlưu ý giúp phụ nữ ngăn ngừa trầm cảm sau sinhTham gia khóa học tiền sảnYêu cầu giúp đỡ từ người thânKhông quá áp lực việc chăm conTrầm cảm sau sinh là gì?'}
[('system', 'Bạn là trợ lý AI hỗ trợ về sức khoẻ tâm lý sau sinh. Dựa vào ngữ cảnh bên dưới hãy đưa ra câu trả lời và lời khuyên phù hợp cho câu hỏi của họ. Nếu bạn không biết câu trả lời, hãy nói không biết, đừng cố tạo ra câu trả lời. \n# Ngữ cảnh: \n{context}'), ('user', '## Câu hỏi: Sơ sinh nghĩa

In [11]:
chain, off_rag = build_rag_chain(llm)
while True:
    question = input("You: ")
    if question == "exit":
        break
    answer = chain.invoke(question)
    for i, (role, text) in enumerate(off_rag.messages):
        if role == "system" and "{context}" in text:
            off_rag.messages[i] = (role, text.replace("{context}", off_rag.list_doc["context"]))
            break
        if role == "user" and "{question}" in text:
            off_rag.messages[i] = (role, text.replace("{question}", question))
    off_rag.messages.append(("assistant", answer))
    off_rag.messages.append(("system", "# Ngữ cảnh: {context}"))
    off_rag.messages.append(("user","## Câu hỏi: {question}\n### Trả lời:"))
    chain = build_rag_chain_from_off_rag(off_rag)
    print(off_rag.messages)

{'context': 'khoẻ sinh sản cho phụ nữ bao gồm cả sàng lọc bạo lực và trầm cảm trong \nmang thai và sau sinh.\n\n- Tiền sử sảy thai \n- Có thai ngoài ý muốn \n- Sinh nhiều con \n \nYẾU TỐ SAU SINH \n- Chế độ nghỉ ngơi, kiêng khem sau \nsinh \n- Giới tính của trẻ \n- Thiếu hỗ trợ gia đình, xã hội sau \nsinh \n- Sinh non \n- Hình thức sinh: sinh mổ/sinh \nthường\n\ncác nhân viên y tế trong thời gian mang thai và sau sinh nhưng họ không tiết\n\nsinh con bác sĩ có thể kê đơn thuốc phòng ngừa trầm cảm sau sinh ngay sau khi em bé chào đời.\n\nlưu ý giúp phụ nữ ngăn ngừa trầm cảm sau sinhTham gia khóa học tiền sảnYêu cầu giúp đỡ từ người thânKhông quá áp lực việc chăm conTrầm cảm sau sinh là gì?'}
[('system', 'Bạn là trợ lý AI hỗ trợ về sức khoẻ tâm lý sau sinh. Dựa vào ngữ cảnh bên dưới hãy đưa ra câu trả lời và lời khuyên phù hợp cho câu hỏi của họ. Nếu bạn không biết câu trả lời, hãy nói không biết, đừng cố tạo ra câu trả lời.\n# Ngữ cảnh: \nkhoẻ sinh sản cho phụ nữ bao gồm cả sàng lọc bạo 